<a href="https://colab.research.google.com/github/sanketp25/QuantumNotes/blob/main/SmartLearn_AI_Speech_to_Text_with_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 84.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2


In [2]:
!pip install torch torchaudio transformers accelerate gradio --quiet


In [3]:
!pip install --upgrade gradio


## Local Inference on GPU
Model page: https://huggingface.co/openai/whisper-small

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/openai/whisper-small)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import torch

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small", device=0 if torch.cuda.is_available() else -1)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [5]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-small")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-small")

In [13]:
import librosa

try:
    from google.colab import files
    colab_env = True
except ImportError:
    colab_env = False

device = "cuda:0" if torch.cuda.is_available() else "cpu"

#function for transcription(chunking for large audio files)
def transcribe_long_audio(file_path, chunk_length_s=30):
    # Load audio at 16kHz (as expected by Whisper)
    audio, sr = librosa.load(file_path, sr=16000)

    # Split into chunks
    chunk_size = chunk_length_s * sr
    num_chunks = (len(audio) // chunk_size) + 1

    transcripts = []

    for i in range(num_chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(audio))
        chunk = audio[start:end]

        if len(chunk) == 0:
            continue

        result = pipe({"array": chunk, "sampling_rate": sr})
        transcripts.append(result["text"])

    return " ".join(transcripts)

In [14]:
from google.colab import files
import os
if colab_env:
    print("Running in Colab – please upload your lecture recording...")
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
else:
    # Default path for local/GitHub runs
    file_name = input("Enter path to audio file: ").strip()
    if not os.path.exists(file_name):
        raise FileNotFoundError(f"File not found: {file_name}")

print("⏳ Transcribing, please wait...")
transcript = transcribe_long_audio(file_name, chunk_length_s=30)

# Save transcript
output_file = "lecture_transcript.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(transcript)

print(f"✅ Transcription completed and saved as {output_file}")

# Download in Colab
if colab_env:
    files.download(output_file)

Running in Colab – please upload your lecture recording...


Saving ana-bell-v1.mp3 to ana-bell-v1.mp3
⏳ Transcribing, please wait...
✅ Transcription completed and saved as lecture_transcript.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>